# <font color="blue"><center>Génération des données</font>
    
## &nbsp;&nbsp;&nbsp;&nbsp;&nbsp;Algorithme
    

In [6]:
import numpy as np
import json
import os
def grapheAleatoire(taille, delivery_window_start):
    data= []
    plage_horaire = {'3h-9h': 25/100, '9h-15h': -25/100, '15h-21h': 30/100,'21h-3h':-50/100} #Tableau avec nos prévisions de trafics
    b = np.random.choice((True, False), size=(taille,taille), p=[0.4, 0.6]) #Génération d'une matrice binaire
    b_symm = np.logical_or(b, b.T) 
    matrice =  b_symm.astype(int) #Rendre la matrice non orienté
    
    #Récupération des graphes présents dans le fichier
    fileName = r"graph.json"
    if os.path.isfile(fileName):
        with open('graph.json', 'r') as fp:
            data = json.load(fp)
    #Ajout des poids dans la matrice en fonction du début de la fenêtre de livraison
    for ligne in range(taille):
        if (matrice[ligne][ligne] == 1):
            matrice[ligne][ligne] = 0
        for sommet in range(taille):            
            if (matrice[ligne][sommet] == 1):
                
                poids = np.random.randint(5,15)
                if (delivery_window_start < 3 or delivery_window_start >= 21):
                    
                    poids += plage_horaire['21h-3h'] * poids
                if (delivery_window_start >= 3 and delivery_window_start < 9):
                    poids += plage_horaire['3h-9h'] * poids
                if (delivery_window_start >= 9 and delivery_window_start < 15):
                    poids += plage_horaire['9h-15h'] * poids
                if (delivery_window_start >= 15 and delivery_window_start < 21):
                    poids += plage_horaire['15h-21h'] * poids
                matrice[ligne][sommet] = poids
                matrice[sommet][ligne] = matrice[ligne][sommet]
    data.append(matrice.tolist())
    #Remplissage du fichier avec le nouveau graphe
    with open('graph.json', 'w') as fp:
        json.dump(data, fp)
 
grapheAleatoire(5, 22)

# <font color="blue"><center>Modélisation du problème</font>

## Définition du problème formel
### &nbsp;&nbsp;&nbsp;&nbsp;&nbsp; Données

Notre problème comportents plusieurs données :
<ul>
    <li>Un graphe pondéré G ==> chaque arête à un poids correspondant au nombre de véhicules circulant sur cette arête et pouvant varié selon l'heure de la journée</li>
    <li>Plusieurs sommets ==> ici, chaque sommet correspond à un sous-ensemble de villes</li>
    <li>Un cycle ==> effectuer une tournée de livraison en partant d'un point et en revenant à ce même point</li>
    <li>Fenêtres de temps de livraison ==> les livraisons devront être effectué dans une plage horaire définis</li>
</ul>

### &nbsp;&nbsp;&nbsp;&nbsp;&nbsp; Question 

Existe-t-il un cycle passant au moins une fois par chaque sommet de G qui respecte les fenêtres de temps de livraison et dont la somme des valeurs des arêtes est au plus k ?

## Etude de complexité de ce problème

### &nbsp;&nbsp;&nbsp;&nbsp;&nbsp;Notre problème est-il dans NP ?
Grâce à l'instance de notre problème, on peut déterminer si oui ou non notre problème est dans NP.
Nous pouvons vérifier l'instance de notre problème en temps polynomial car pour un cycle, on peut déterminer si:
    
<ul>
    <li>Il s'agit bien d'un cycle ==> vérifier que la première et la dernière valeur sont le même sommet</li>
    <li>Le cycle passe bien par tous les sommets ==> parcourir le tableau en s'assurant que tous les sommets sont bien présents</li>
    <li>Son coût est inférieur à k ==> comparé le poids du chemin obtenu avec k</li>
</ul>

Notre problème est donc dans NP.
    
### &nbsp;&nbsp;&nbsp;&nbsp;&nbsp;Est-il NP Complet ou non ?

Notre instance de problème I est une instance red(I) du problème du voyageur de commerce (TSP) car le voyageur de commerce consiste à chercher un cycle hamiltonien de plus petit poids dans un graphe pondéré complet (ce n'est pas notre cas). Généralement, le poids correspond à la distance de l'arête mais il est possible d'adapter cette représentation à notre cas de figure.
    
*Cycle hamiltonien : chemin qui passe au moins une fois par chacun des sommets en revenant à son point de départ.*
<br>*Graphe complet : graphe simple dont tous les sommets sont adjacents deux à deux, c'est-à-dire que tout couple de sommets disjoints est relié par une arête*
![graphe](https://i1.wp.com/www.methodemaths.fr/graphe_complet.jpg?w=584&ssl=1)
Réduction polynomiale :
    <ul>
        <li>Notre instance I (G = (S,A),k) et l'instance TSP red(I) (G' = (S,A'),k) ==> Pour tout (G,k), I(G,k) si et seulement si red(I)(G',k) ?</li> Si (G = (S, A), k) a un chemin C de poids inférieur à k, C est un chemin de poids inférieur à k de G'. <br>Réciproquement: si (G' = (S, A' ), k) a un chemin C de poids inférieur à k, C est un chemin de poids inférieur à k de G.<br><br>
    <li>La transformation est polynomiale: l’algorithme qui calcule red(I) est bien polynomial : sa complexité est de n²2^n ( voir <a href="http://algorithmics.lsi.upc.edu/docs/Dasgupta-Papadimitriou-Vazirani.pdf">Algorithms</a>
, p179)</li><br>
On peut donc dire que notre problème est au moins aussi difficile que le problème du voyageur de commerce, soit notre problème P >= TSP. 
Cependant, on sait que le problème TSP est NP difficile et notre problème P est dans NP, donc d'après le schéma ci-dessous, notre problème est NP Complet.

<img src="https://4.bp.blogspot.com/-gxblHm308a8/XI-3wxvpWUI/AAAAAAAADKc/4u5HTH2SLkcoj1oiyPW8bUuMEjCPOQNKwCLcBGAs/s1600/Diff%25C3%25A9rence%2Bentre%2Bun%2Bprobl%25C3%25A8me%2BNP-Complet%2Bet%2BNP-Difficile-1.png" alt="drawing"/>

# <font color="blue"><center>Modélisation de l'algorithme (tournée du voyageur de commerce)</font>

## Principe de l'algorithme
Les algorithmes de colonies de fourmis sont des algorithmes appartenant à une discipline mathématique et informatique qui étudie les graphes. Cette discipline est nommée « la théorie des graphes ». 
Les algorithmes de colonies de fourmis sont des algorithmes qui réutilise les comportements des fourmis. Ils ont été créer dans les années 1990, par Marco Dorigo dans le but de résoudre des problèmes comme le voyageur de commerce. 

Principe de l’algorithme :
Les fourmis suivent des étapes définies :
1.    une fourmi parcourt plus ou moins au hasard l’environnement autour de la colonie
2.    si celle-ci découvre une source de nourriture, elle rentre plus ou moins directement au nid, en laissant sur son chemin une piste de phéromones
3.    ces phéromones étant attractives, les fourmis passant à proximité vont avoir tendance à suivre, de façon plus ou moins directe, cette piste
4.    en revenant au nid, ces mêmes fourmis vont renforcer la piste
5.    si deux pistes sont possibles pour atteindre la même source de nourriture, celle étant la plus courte sera, dans le même temps, parcourue par plus de fourmis que la longue piste
6.    la piste courte sera donc de plus en plus renforcée, et donc de plus en plus attractive
7.    la longue piste, elle, finira par disparaître, les phéromones étant volatiles
8.    à terme, l’ensemble des fourmis a donc déterminé et « choisi » la piste la plus courte.

Prenons comme exemple un ensemble de villes. Le but est de trouver le plus court chemin passant une seul et uniques fois par toutes ces villes tout en revenant au point de départ. 
Une fourmi part de la ville V0. Son prochain choix sera déterminé par la distance entre les villes mais aussi par la quantité de phéromones de fourmi. Ensuite elle réitère le choix jusqu’au moments où il ne reste plus de ville et donc elle revient à la ville de départ V0. Si le chemin est long alors les autre fourmi essayerons de trouver un autre chemin plus court.

## Algorithme : colonies de fourmis

#### Données: 

* n est le nombre de fourmis parcourant les villes
* L contient les villes à parcourir avec des informations entre les villes
* j le nombre d'itération du programme

#### Resultat: 
* D Liste qui contient la liste du poarcours de villes à effectuer

#### Pseudo code 
<blockquote>
    debut <br />
        &nbsp;&nbsp;&nbsp;&nbsp;A : Choisir ville de départ L<br />
        &nbsp;&nbsp;&nbsp;&nbsp;P : Initialisation des phéromones<br /><br />
        &nbsp;&nbsp;&nbsp;&nbsp;pour m allant de 1 à n faire<br />
        &nbsp;&nbsp;&nbsp;&nbsp;&nbsp;liste : L<br /><br />
        &nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;pour i allant de 1 à n faire<br />
        &nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;liste : enlever ville (A, liste)<br />
        &nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;B : A<br /><br />
        &nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;tant que L non vide faire<br />
        &nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;B: Choisir ville suivante(liste)<br /><br />
        &nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;Si B est dans la bonne tranche horaire<br />
        &nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;liste: enlever ville (B, liste)<br /><br />
        &nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp; Sinon attendre future tranche horaire<br /><br />
        &nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;fin<br />
        &nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;P : déposer phéronmones P, évaporation phéroomones<br /><br />
        &nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;fin<br />
        &nbsp;&nbsp;&nbsp;&nbsp;fin<br />
        &nbsp;&nbsp;&nbsp;&nbsp;Rendre D = derniere fourmi P<br /><br />
    fin
    <br />
</blockquote>


## Modélisation linéaire : Simplex Algorithm
<blockquote>

### Variables de décision
<br />
     $c_(ij)$ est le coût de l'arête allant de i à j. <br /><br />
     $x_(ij) = $$\begin{pmatrix} 1 & Si&j&est&visité&apres&i \\ 0 & Sinon \end{pmatrix}$$ $<br /><br />
     $x_(ij) = 1$ si la tournée va du sommet $i$ au sommet $j$, et 0 sinon. Le tour doit avoir exactement une arête sortant du sommet $i$ (degré de sortie = 1) et une arête entrant dans le sommet $i$ (degré d'entrée = 1). Cela s'explique par le fait qu'un tour est un cycle.<br /><br />


 $u(i)$ et $u(j)$ sont des variables fictives pour lesquelles nous devons résoudre.<br /><br />
    
### Contraintes du programme
* Contrainte 1 : Besoins d'avoir que chaque ville ait exactement un successeur dans la tournée : $\displaystyle\sum_{ j | (i,j) \in[A]}{X(ij)} = 1 $
<br /><br />

* Contrainte 2 : Besoins d'avoir exactement un prédécesseur pour chaque noeud :$\displaystyle\sum_{ i | (i,j) \in[A]}{X(ij)} = 1$
<br /><br />
    
* Contrainte 3 : Élimination des sous-tours :     $u(i)$-$u(j)$&nbsp; + &nbsp; $ $n$x_(ij)$$\leq$n$ $ &nbsp; - &nbsp; $1$
<br /><br />
    
### Objectif fonction
<br />
   <p>Nous voulons minimiser la somme des connections entre les villes du cout entre $i$ à $j$. Puisque nous avons une fonction linéaire que nous souhaitons optimiser et un ensemble de contraintes, nous pouvons appliquer la méthode du simplexe pour résoudre cette forme de programme linéaire.</p>

<br />

### Programme linéaire
<br />
    $\text{Minimiser  $z$ = } \displaystyle{n}\sum_{i=1}\displaystyle{n}\sum_{j=1}{C(ij)}X(ij) $
<br /><br />
    $\text{sc to}$ &nbsp; $\displaystyle{n}\sum_{j=i}{X(ij)} = 1$&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp; $1\leq(i)\leq(n)$ &nbsp;&nbsp;&nbsp;$ ∀(i)\in(I) $
<br /><br />
    $\displaystyle{n}\sum_{i=j}{X(ij)} = 1$&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp; $1\leq(j)\leq(n)$ &nbsp;&nbsp;&nbsp;$ ∀(i)\in(J) $
<br /><br />
    $u(i)$-$u(j)$&nbsp; + &nbsp; $ $n$x_(ij)$$\leq$n$ $ &nbsp; - &nbsp; $1$


</blockquote><br /><br />